In [4]:
def convert_to_gemini_schema(pydantic_schema: Dict[str, Any]) -> Dict[str, Any]:
    gemini_schema = {}
    
    # Handle Optional types (which Pydantic often represents with anyOf)
    if "anyOf" in pydantic_schema:
        # Check if one of the options is "null" (which indicates Optional)
        null_present = any(schema.get("type") == "null" for schema in pydantic_schema["anyOf"])
        
        # If null is one of the options, it's an Optional type
        if null_present:
            # Find the non-null type and use it as the base
            for schema in pydantic_schema["anyOf"]:
                if schema.get("type") != "null":
                    # Save the description before processing the non-null schema
                    description = pydantic_schema.get("description")
                    
                    # Use the non-null schema as our base
                    non_null_schema = convert_to_gemini_schema(schema)
                    gemini_schema.update(non_null_schema)
                    gemini_schema["nullable"] = True
                    
                    # Restore the description if it exists
                    if description:
                        gemini_schema["description"] = description
                    
                    return gemini_schema
    
    # Map basic type
    if "type" in pydantic_schema:
        # Convert Pydantic types to Gemini types
        type_mapping = {
            "string": "STRING",
            "number": "NUMBER",
            "integer": "INTEGER",
            "boolean": "BOOLEAN",
            "array": "ARRAY",
            "object": "OBJECT"
        }
        gemini_schema["type"] = type_mapping.get(pydantic_schema["type"], pydantic_schema["type"])
    
    # Map format if available
    if "format" in pydantic_schema:
        gemini_schema["format"] = pydantic_schema["format"]
    
    # Map description
    if "description" in pydantic_schema:
        gemini_schema["description"] = pydantic_schema["description"]
    
    # Map enum values
    if "enum" in pydantic_schema:
        gemini_schema["enum"] = pydantic_schema["enum"]
    
    # Map array constraints
    if "maxItems" in pydantic_schema:
        gemini_schema["maxItems"] = str(pydantic_schema["maxItems"])
    if "minItems" in pydantic_schema:
        gemini_schema["minItems"] = str(pydantic_schema["minItems"])
    
    # Map properties
    if "properties" in pydantic_schema:
        gemini_schema["properties"] = {}
        for prop_name, prop_schema in pydantic_schema["properties"].items():
            gemini_schema["properties"][prop_name] = convert_to_gemini_schema(prop_schema)
    
    # Map required fields
    if "required" in pydantic_schema:
        gemini_schema["required"] = pydantic_schema["required"]
    
    # Map items for arrays
    if "items" in pydantic_schema:
        gemini_schema["items"] = convert_to_gemini_schema(pydantic_schema["items"])
    
    # Add propertyOrdering if needed
    if "properties" in pydantic_schema:
        gemini_schema["propertyOrdering"] = list(pydantic_schema["properties"].keys())
    
    return gemini_schema